# Agentic Frameworks

## LlamaIndex

### Librerias


In [ ]:

%pip install llama-index
%pip install llama-index-llms-azure-openai

In [2]:
import os
import requests

from dotenv import load_dotenv
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.core.tools import FunctionTool
from llama_index.llms.azure_openai import AzureOpenAI

# Carga de variables de .env

In [3]:
load_dotenv(override=True)

az_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_URL")
az_openai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
az_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
bing_api_key = os.getenv("BING_API_KEY")

# Crea el cliente Azure OpenAI en LlamaIndex

In [4]:
llm = AzureOpenAI(
    engine=az_openai_deployment_name,
    azure_endpoint=az_openai_endpoint,
    api_key=az_openai_api_key,
    api_version="2024-05-01-preview",
)

# Añadimos Tools - Busqueda en Bing

In [5]:
def search_for_data_in_bing(query):
    """Search the Bing Web Search API and return the first result."""
    headers = {
        "Ocp-Apim-Subscription-Key": bing_api_key
    }
    params = {
        "q": query,
        "count": 1  # Only request 1 result
    }

    response = requests.get("https://api.bing.microsoft.com/v7.0/search", headers=headers, params=params)
    response.raise_for_status()
    data = response.json()

    if "webPages" in data and "value" in data["webPages"] and len(data["webPages"]["value"]) > 0:
        first_result = data["webPages"]["value"][0]
        snippet = first_result.get("snippet", "No Snippet")
        return snippet
    else:
        return "No results found."

## Probamos la tool

In [ ]:
search_for_data_in_bing("Capital de Estados Unidos")

## Registro de tools para llamada a Azure OpenAI

In [7]:
search_tool = FunctionTool.from_defaults(fn=search_for_data_in_bing)

## Llamada a API con tools

In [8]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [search_tool], 
    llm=llm,
    verbose=False
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Quien ganó la eurocopa de 2024?"
)

print(f" La respuesta del LLM es {response}")